In [1]:
import os

In [2]:
path_img_106 = []
path_img_ngt = []

In [3]:
root = r"C:\Users\lucyc\Desktop\DatSet_Activate\105_face"
dir_list = os.listdir(root)
for dir_person in dir_list:
    path_img_106.append([root+"\\"+dir_person+"\\"+x for x in os.listdir(root + "\\" + dir_person)])

root = r"C:\Users\lucyc\Desktop\DatSet_Activate\diff_face"
path_img_ngt = [root+"\\"+x for x in os.listdir(root)[:18000]]

In [4]:
path_data_trp = []

In [5]:
count = 0
for cla_imgs in path_img_106:
    for i in range(1, len(cla_imgs), 2):
        path_data_trp.append((cla_imgs[i], cla_imgs[i-1], path_img_ngt[count]))
        count += 1

In [6]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [7]:
def img_load_preprocess(paths, usl):
    images = []
    for path in paths:
        # 加载图片
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        
        # 应用其他预处理步骤...
        image = tf.image.resize_with_pad(
                            image,
                            target_height=224,
                            target_width=224,
                            method=tf.image.ResizeMethod.BILINEAR
                        )
        image = tf.cast(image, tf.float32) / 255.0
        images.append(image)
    
    # 返回配对的图片
    return (tuple(images), usl)

In [8]:
class TripletLoss(keras.metrics.Metric):
    
    def __init__(self, name="trip", **kwargs):
        super().__init__(name=name, **kwargs)
        # self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        # self.total_samples = self.add_weight(
        # name="total_samples", initializer="zeros", dtype="int32")
        self.positive_dis = self.add_weight(name="positive_dis", initializer="zeros")
        self.negative_dis = self.add_weight(name="negative_dis", initializer="zeros")

    def update_state(self, anchor, positive, negative):
        # y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        # mse = tf.reduce_sum(tf.square(y_true - y_pred))
        # self.mse_sum.assign_add(mse)
        # self.total_samples.assign_add(num_samples)
        positive_dis = tf.norm(anchor - positive, axis=-1)
        negative_dis = tf.norm(anchor - negative, axis=-1)
        self.positive_dis.assign(positive_dis)
        self.negative_dis.assign(negative_dis)
    
    def result(self):
        return self.positive_dis, self.negative_dis

    def reset_state(self):
        self.positive_dis.assign(0)
        self.negative_dis.assign(0)


def loss_fun(anchor, positive, negative):
    positive_dis = tf.norm(anchor - positive, axis=-1)
    negative_dis = tf.norm(anchor - negative, axis=-1)
    return positive_dis, negative_dis

In [9]:
class SiameseCNN(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv2D(filters=32, kernel_size=3, activation="relu")
        self.conv2 = layers.Conv2D(filters=64, kernel_size=3, activation="relu")
        self.conv3 = layers.Conv2D(filters=128, kernel_size=3, activation="relu")
        self.conv4 = layers.Conv2D(filters=128, kernel_size=3, activation="relu")
        self.flatten = layers.Flatten()
        self.dropout = layers.Dropout(0.5)
        self.maxpo = layers.MaxPooling2D(pool_size=2)
        #self.dense = layers.Dense(1, activation="sigmoid")
        
    def call(self, inputs):
        x = self.conv1(inputs) # test here
        x = self.maxpo(x)
        x = self.conv2(x)
        x = self.maxpo(x)
        x = self.conv3(x)
        x = self.maxpo(x)
        x = self.conv4(x)
        return x

    def train_step(self, data):
        anchor = data[0]
        positive = data[1]
        negative = data[2]
        with tf.GradientTape() as tape:
            p_anchor = call(anchor)
            p_positive = call(positive)
            p_negative = call(negative)
            lossp, lossn = loss_fun(p_anchor, p_positive, p_negative)
        gradients_p = tape.gradient(lossp, self.trainable_weights)
        gradients_n = tape.gradient(lossn, self.trainable_weights)
        
        self.optimizer.apply_gradients(zip(gradients_p, self.trainable_weights))
        self.optimizer.apply_gradients(zip(gradients_n, self.trainable_weights))
        
        self.compiled_metrics.update_state(anchor, positive, negative)
        return {m.name: m.result() for m in self.metrics}


In [10]:
model = SiameseCNN()
model.compile(optimizer=keras.optimizers.RMSprop(),
loss=loss_fun,
metrics=[TripletLoss()])


In [11]:
usl_labble = tf.zeros(8740)

In [12]:
# 从张量创建数据集
dataset = tf.data.Dataset.from_tensor_slices((path_data_trp, usl_labble))
dataset = dataset.map(img_load_preprocess)
# 打乱数据并设置批次大小
dataset = dataset.shuffle(buffer_size=64).batch(32)

InaccessibleTensorError: The tensor 'Tensor("while/truediv:0", shape=(224, 224, 3), dtype=float32)' cannot be accessed here: it is defined in another function or code block. Use return values, explicit Python locals or TensorFlow collections to access it. Defined in: FuncGraph(name=while_body_48, id=2895786655168); accessed from: FuncGraph(name=Dataset_map_img_load_preprocess, id=2895783467472).


In [ ]:
model.fit(train_images, train_labels, epochs=3)

In [111]:
#@tf.function

In [103]:
model = SiameseCNN()

In [104]:
img = img_load_preprocess(path_data_trp[0])[0]

In [105]:
img = tf.expand_dims(img, axis=0)

In [106]:
x = model(img)